In [19]:
import numpy as np
import pandas as pd
import matplotlib as plt
import statsmodels.api as sm
import operator
import stat
from numba import jit

In [20]:
EWR = pd.read_excel("Equal Weighted Return 49 industry.xlsx")
VBR = pd.read_excel("Value Based Return 49 industry.xlsx")
RF = pd.read_excel("RF.xlsx")

In [21]:
EWR.replace(-99.99,np.NaN,inplace=True)
VBR.replace(-99.99,np.NaN,inplace=True)

In [22]:
EWR.rename(columns={"Unnamed: 0":"Date"},inplace=True)
VBR.rename(columns={"Unnamed: 0":"Date"},inplace=True)

In [23]:
EWR.set_index("Date",inplace=True)
VBR.set_index("Date",inplace=True)

In [24]:
def Excess(df,rf):
    Excess = pd.DataFrame()
    RF = list(rf.iloc[:,0])
    for i in range(len(df.columns)):
        Rlist = list(df.iloc[:,i])
        Excesslist = []
        for j in range(len(Rlist)):
            Excesslist.append(Rlist[j]-RF[j])
        Excess[df.columns[i]]=Excesslist
    return Excess

In [25]:
Excess_EWR = Excess(EWR,RF)
Excess_VBR = Excess(VBR,RF)

In [26]:
Droplist = ['Soda ','Hlth ','Rubbr','FabPr','Guns ','Gold ','PerSv','Softw','Paper']
Excess_EWR.drop(columns=Droplist,inplace=True)
Excess_VBR.drop(columns=Droplist,inplace=True)

In [27]:
Excess_EWR.describe()
Excess_VBR.describe()

,Agric,Food,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,MedEq,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
count,24998.000000,24998.000000,24998.000000,24998.000000,24998.000000,24998.000000,24998.000000,24998.000000,24998.000000,24998.000000,...,24998.000000,24998.000000,24998.000000,24998.000000,24998.000000,24998.000000,24998.000000,24998.000000,24998.000000,24998.000000
mean,0.031065,0.031143,0.041901,0.039414,0.037786,0.042282,0.031613,0.029414,0.029044,0.042640,...,0.036196,0.028900,0.028093,0.034758,0.036139,0.039201,0.033453,0.026134,0.036384,0.020385
std,1.513487,0.930707,1.453964,1.210530,2.148579,1.802883,1.569168,1.163920,1.167340,1.588161,...,1.266883,1.360495,1.652707,1.141450,1.342087,1.486611,1.377363,2.137288,1.576160,1.484566
min,-15.274000,-16.067000,-24.061000,-14.017000,-26.755000,-24.137000,-19.367000,-21.487000,-18.537000,-53.625000,...,-21.457000,-17.587000,-44.441000,-18.037000,-15.507000,-20.448000,-17.154000,-21.230000,-28.650000,-20.261000
25%,-0.659000,-0.361000,-0.573750,-0.490750,-0.864000,-0.737000,-0.652000,-0.484000,-0.466000,-0.625000,...,-0.571000,-0.578000,-0.552000,-0.454000,-0.604000,-0.518000,-0.543000,-0.800000,-0.554000,-0.630000
50%,0.003000,0.048000,0.030000,0.030000,-0.001000,0.034000,0.011000,0.030500,0.042000,0.029000,...,0.040000,0.030000,0.042000,0.050000,0.049000,0.034000,0.031500,-0.002000,0.040000,0.026000
75%,0.710000,0.446000,0.653000,0.569750,0.900000,0.840000,0.690000,0.548000,0.542000,0.709000,...,0.648000,0.645750,0.628000,0.530000,0.683750,0.600000,0.609000,0.799000,0.649000,0.682000
max,23.688000,15.538000,19.908000,16.202000,39.738000,20.806000,33.398000,25.868000,20.484000,111.815000,...,12.588000,18.488000,66.919000,17.808000,19.398000,19.042000,18.929000,36.778000,23.262000,16.814000


In [28]:
def Price(df):
    Price = pd.DataFrame()
    for i in range(len(df.columns)):
        Rlist = list(df.iloc[:,i])
        Plist = [1]
        for j in range(len(Rlist)):
            Plist.append(Plist[-1]*(1+(Rlist[j]/100)))
        Price[df.columns[i]]=Plist
    return Price

In [29]:
EWP = Price(Excess_EWR)
VBP = Price(Excess_VBR)

In [30]:
def EMA(df,decay):
    EMA = pd.DataFrame()
    Nday = int((2/decay)-1)
    for i in range(len(df.columns)):
        Plist = list(df.iloc[:,i])
        count = 0
        Null_count = 0
        AVGlist = []
        for n in range(len(Plist)):
            if count==Nday:
                break
            elif pd.isnull(Plist[n])==True:
                Null_count+=1
            else: 
                count+=1
                AVGlist.append(Plist[n])
        EMAlist = [np.NaN]*(Nday+Null_count)    
        EMAlist.append(np.average(AVGlist))
        for j in range(len(Plist)-Nday-1-Null_count):
            EMAlist.append(decay*Plist[j+Nday+1+Null_count]+(1-decay)*EMAlist[-1])
        EMA[df.columns[i]]=EMAlist
    return EMA

In [31]:
def Dif(df1,df2):
    Dif = pd.DataFrame()
    for i in range(len(df1.columns)):
        df1list = list(df1.iloc[:,i])
        df2list = list(df2.iloc[:,i])
        Diflist = []
        for j in range(len(df1list)):
            Diff = df1list[j]-df2list[j]
            Diflist.append(Diff)   
        Dif[df1.columns[i]]=Diflist
    return Dif

In [32]:
def Simple_Cross(Histogram,Return):
    SCPdf = pd.DataFrame()
    Returndf = pd.DataFrame()
    for j in range(len(Histogram.columns)):
        Simple_Cross_Price = 1
        SCPlist = []
        Returnlist = []
        for i in range (len(Histogram)-1):
            if pd.isnull(Histogram.iloc[i,j])==True:
                SCPlist.append(np.NaN)
                Returnlist.append(np.NaN)
            else:
                if Histogram.iloc[i,j]>=0:
                    SCPlist.append(Simple_Cross_Price)
                    Returnlist.append(Return.iloc[i,j])
                    Simple_Cross_Price=(1+(Return.iloc[i,j]*(1)/100))*Simple_Cross_Price
                else:
                    SCPlist.append(Simple_Cross_Price)
                    Returnlist.append(Return.iloc[i,j]*(-1))
                    Simple_Cross_Price=(1+(Return.iloc[i,j]*(-1)/100))*Simple_Cross_Price 
        SCPdf[Histogram.columns[j]] = SCPlist
        Returndf[Histogram.columns[j]] = Returnlist
    return SCPdf,Returndf

In [33]:
def Sign(n):
    if n>=0:
        return ("P")
    else:
        return ("N")

In [34]:
def N_day_hold(Histogram,Return):
    SCPdf = pd.DataFrame()
    Returndf = pd.DataFrame()
    for j in range(len(Histogram.columns)):
        Simple_Cross_Price = 1
        SCPlist = []
        Returnlist = []
        for n in range (len(Histogram)):
            if pd.isnull(Histogram.iloc[n,j])==False:
                if Sign(Histogram.iloc[n,j])=='P':
                    Long_Term = 'L'
                    break
                else:
                    Long_Term = 'S'
                    break        
        for i in range (len(Histogram)-1):
            if pd.isnull(Histogram.iloc[i,j])==True:
                SCPlist.append(np.NaN)
                Returnlist.append(np.NaN)
            else:
                if pd.isnull(Histogram.iloc[i-1,j])==True:
                    if Histogram.iloc[i,j]>=0:
                        SCPlist.append(Simple_Cross_Price)
                        Returnlist.append(Return.iloc[i,j])
                        Simple_Cross_Price=(1+(Return.iloc[i,j]*(1)/100))*Simple_Cross_Price
                    else:
                        SCPlist.append(Simple_Cross_Price)
                        Returnlist.append(Return.iloc[i,j]*(-1))
                        Simple_Cross_Price=(1+(Return.iloc[i,j]*(-1)/100))*Simple_Cross_Price
                else:
                    if pd.isnull(Histogram.iloc[i-2,j])==True:
                        if Long_Term == 'L':
                            SCPlist.append(Simple_Cross_Price)
                            Returnlist.append(Return.iloc[i,j])
                            Simple_Cross_Price=(1+(Return.iloc[i,j]*(1)/100))*Simple_Cross_Price
                        else:
                            SCPlist.append(Simple_Cross_Price)
                            Returnlist.append(Return.iloc[i,j]*(-1))
                            Simple_Cross_Price=(1+(Return.iloc[i,j]*(-1)/100))*Simple_Cross_Price
                    elif (Sign(Histogram.iloc[i,j])==Sign(Histogram.iloc[i-1,j]) and Sign(Histogram.iloc[i,j])==Sign(Histogram.iloc[i-2,j]))\
                    or (Sign(Histogram.iloc[i,j])!=Sign(Histogram.iloc[i-1,j]) and Sign(Histogram.iloc[i,j])==Sign(Histogram.iloc[i-2,j]))\
                    or (Sign(Histogram.iloc[i,j])!=Sign(Histogram.iloc[i-1,j]) and Sign(Histogram.iloc[i,j])!=Sign(Histogram.iloc[i-2,j])):
                        if Long_Term == 'L':
                            SCPlist.append(Simple_Cross_Price)
                            Returnlist.append(Return.iloc[i,j])
                            Simple_Cross_Price=(1+(Return.iloc[i,j]*(1)/100))*Simple_Cross_Price
                        else:
                            SCPlist.append(Simple_Cross_Price)
                            Returnlist.append(Return.iloc[i,j]*(-1))
                            Simple_Cross_Price=(1+(Return.iloc[i,j]*(-1)/100))*Simple_Cross_Price
                    elif (Sign(Histogram.iloc[i,j])==Sign(Histogram.iloc[i-1,j]) and Sign(Histogram.iloc[i,j])!=Sign(Histogram.iloc[i-2,j])):
                        if Long_Term == 'L':
                            Lone_Term = 'S'
                            SCPlist.append(Simple_Cross_Price)
                            Returnlist.append(Return.iloc[i,j]*(-1))
                            Simple_Cross_Price=(1+(Return.iloc[i,j]*(-1)/100))*Simple_Cross_Price
                        else:
                            Lone_Term = 'L'
                            SCPlist.append(Simple_Cross_Price)
                            Returnlist.append(Return.iloc[i,j])
                            Simple_Cross_Price=(1+(Return.iloc[i,j]*(1)/100))*Simple_Cross_Price 
        SCPdf[Histogram.columns[j]] = SCPlist
        Returndf[Histogram.columns[j]] = Returnlist
    return SCPdf,Returndf

In [35]:
def Alpha(MarketR,StrategyR):
    summary_dict = {}
    for i in range(len(MarketR.columns)):
        Market_ER = list(Excess_EWR.iloc[:,i])
        Strategy_ER = list(StrategyR.iloc[:,i])
        count = 0
        for j in range(len(Strategy_ER)):
            if pd.isnull(Strategy_ER[j])==True:
                count+=1
        Market_ER = Market_ER[count:]
        Strategy_ER = Strategy_ER[count:]
        X = []
        Y = []
        for n in range(len(Market_ER)):
            X.append([Market_ER[n]])
            Y.append([Strategy_ER[n]])
        X = np.array(X)
        X = sm.add_constant(X.ravel())
        model = sm.OLS(Y,X).fit()
        pvalue = model.pvalues
        param = model.params
        summary = model.summary()
        summary_dict[MarketR.columns[i]]=[pvalue,param,summary]
    return summary_dict

In [20]:
A = [0.01*(n+1) for n in range(0,20)]
B = A
C = A
Result_Dict = {}
for b in B:
    EMA_b = EMA(EWP,b)
    for a in A:
        EMA_a = EMA(EWP,a)
        Dif_ab = Dif(EMA_a,EMA_b)
        for c in C:
            if c>a and a>b:
                Result = Simple_Cross(Dif(Dif_ab,EMA(Dif_ab,c)),Excess_EWR)
                Result_Dict[(a,b,c)]= Result
Alpha_Dict = {}
Beta_Dict = {}
for i in (Result_Dict.keys()):
    Alpha_Result = Alpha(Excess_EWR,Result_Dict[i][1])
    Alpha_list = []
    Beta_list = []
    for j in range(40):
        if (Alpha_Result[Excess_EWR.columns[j]][0][0])<0.05:
            Alpha_list.append(Alpha_Result[Excess_EWR.columns[j]][1][0])
            Beta_list.append(Alpha_Result[Excess_EWR.columns[j]][1][1])
        else:
            Alpha_list.append(0)
            Beta_list.append(0)
    Alpha_Dict[i]=Alpha_list
    Beta_Dict[i]=Beta_list
Optimal_params = {}
for n in range(40):
    Alpha_param = {}
    Beta_param = {}
    for p in (Alpha_Dict.keys()):
        Alpha_param[p]=Alpha_Dict[p][n]
        Beta_param[p]=Beta_Dict[p][n]
    Optimal_param = max(Alpha_param.items(), key=operator.itemgetter(1))[0]
    Optimal_Alpha = Alpha_param[Optimal_param]
    Optimal_Beta = Beta_param[Optimal_param]
    Optimal_params[Excess_EWR.columns[n]] = [Optimal_param,[Optimal_Alpha,Optimal_Beta]]

In [21]:
Optimal_params

{'Agric': [(0.02, 0.01, 0.03), [0, 0]],
 'Food ': [(0.19, 0.01, 0.2), [0.07232431959663892, -0.19483059473793174]],
 'Beer ': [(0.17, 0.1, 0.19), [0.039098938252317056, -0.11380870927662443]],
 'Smoke': [(0.14, 0.06, 0.15), [0.04761803208070661, -0.050879498486672274]],
 'Toys ': [(0.17, 0.01, 0.19), [0.0683629537158268, -0.19286900396142567]],
 'Fun  ': [(0.05, 0.01, 0.18), [0.11206588229575605, -0.15306254507653905]],
 'Books': [(0.19, 0.04, 0.2), [0.06966339253582347, -0.04473023755007038]],
 'Hshld': [(0.15, 0.01, 0.2), [0.1103934824417164, -0.21457558287394501]],
 'Clths': [(0.15, 0.01, 0.18), [0.09973979614310911, -0.09687712166843565]],
 'MedEq': [(0.19, 0.02, 0.2), [0.09378228137290108, -0.1876334703796208]],
 'Drugs': [(0.18, 0.04, 0.19), [0.11147621625741963, -0.18472900188255292]],
 'Chems': [(0.19, 0.04, 0.2), [0.0938334977351074, -0.20592007407886545]],
 'Txtls': [(0.18, 0.01, 0.2), [0.11003499508062833, -0.14753578265825748]],
 'BldMt': [(0.19, 0.04, 0.2), [0.104899556852

In [36]:
A = [0.01*(n+1) for n in range(0,20)]
B = A
C = A
Result_Dict = {}
for b in B:
    EMA_b = EMA(VBP,b)
    for a in A:
        EMA_a = EMA(VBP,a)
        Dif_ab = Dif(EMA_a,EMA_b)
        for c in C:
            if c>a and a>b:
                Result = Simple_Cross(Dif(Dif_ab,EMA(Dif_ab,c)),Excess_VBR)
                Result_Dict[(a,b,c)]= Result
Alpha_Dict = {}
Beta_Dict = {}
for i in (Result_Dict.keys()):
    Alpha_Result = Alpha(Excess_VBR,Result_Dict[i][1])
    Alpha_list = []
    Beta_list = []
    for j in range(40):
        if (Alpha_Result[Excess_VBR.columns[j]][0][0])<0.05:
            Alpha_list.append(Alpha_Result[Excess_VBR.columns[j]][1][0])
            Beta_list.append(Alpha_Result[Excess_VBR.columns[j]][1][1])
        else:
            Alpha_list.append(0)
            Beta_list.append(0)
    Alpha_Dict[i]=Alpha_list
    Beta_Dict[i]=Beta_list
Optimal_params = {}
for n in range(40):
    Alpha_param = {}
    Beta_param = {}
    for p in (Alpha_Dict.keys()):
        Alpha_param[p]=Alpha_Dict[p][n]
        Beta_param[p]=Beta_Dict[p][n]
    Optimal_param = max(Alpha_param.items(), key=operator.itemgetter(1))[0]
    Optimal_Alpha = Alpha_param[Optimal_param]
    Optimal_Beta = Beta_param[Optimal_param]
    Optimal_params[Excess_VBR.columns[n]] = [Optimal_param,[Optimal_Alpha,Optimal_Beta]]

In [37]:
Optimal_params

{'Agric': [(0.17, 0.15, 0.19), [0.029904019954578698, -0.027090355998141726]],
 'Food ': [(0.19, 0.01, 0.2), [0.03939458341990678, -0.2256261632755183]],
 'Beer ': [(0.16, 0.13, 0.19), [0.036404786256280526, -0.11409986145082238]],
 'Smoke': [(0.17, 0.02, 0.19), [0.03767070001145005, -0.06160565740778799]],
 'Toys ': [(0.05, 0.02, 0.14), [0.04485647555351231, -0.12891020980008416]],
 'Fun  ': [(0.16, 0.01, 0.2), [0.07241042988901893, -0.13303082222249946]],
 'Books': [(0.16, 0.08, 0.2), [0.0437263055195454, -0.09232788698940844]],
 'Hshld': [(0.13, 0.05, 0.18), [0.03472623845823429, -0.17921500094318937]],
 'Clths': [(0.12, 0.01, 0.19), [0.05871076899430786, -0.12556614115805292]],
 'MedEq': [(0.17, 0.01, 0.2), [0.028871271345538657, -0.2239499368369492]],
 'Drugs': [(0.13, 0.12, 0.18), [0.040650012130527574, -0.1526312035318588]],
 'Chems': [(0.18, 0.04, 0.19), [0.05088232662749538, -0.19642141262902563]],
 'Txtls': [(0.19, 0.01, 0.2), [0.08455376657479753, -0.16503139496435462]],
 'B